<a href="https://colab.research.google.com/github/rendyleo/Program-Crawling/blob/main/Crawling%20Berita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Crawling adalah proses otomatis untuk mengumpulkan atau menelusuri halaman-halaman web di internet menggunakan program khusus yang disebut web crawler atau spider. Web crawler secara sistematis mengikuti tautan dari satu halaman ke halaman lain, mengunduh dan menyimpan konten halaman tersebut untuk keperluan analisis, pengindeksan, atau pengumpulan data.

Proses crawling biasanya dilakukan oleh mesin pencari seperti Google, Bing, atau Yahoo! untuk mengindeks halaman web sehingga pengguna bisa menemukan informasi melalui hasil pencarian. Namun, crawling juga dapat digunakan dalam banyak aplikasi lain, seperti riset pasar, analisis web, atau pengumpulan data yang terstruktur dari web.

 Cara Kerja Crawling:
1. Starting Point: Crawler memulai dari satu atau beberapa URL yang disebut "seed" (awal penjelajahan).
2. Menjelajahi Link: Crawler menelusuri halaman yang ditemukan dan mengumpulkan semua tautan yang ada di halaman tersebut.
3. Mengunduh Halaman: Halaman web diunduh dan disimpan oleh crawler.
4. Pengulangan: Crawler kemudian mengikuti tautan yang ditemukan pada halaman-halaman yang telah dikunjungi, dan proses ini terus berlanjut secara rekursif.

 Kegunaan Crawling:
- Pengindeksan Mesin Pencari: Mesin pencari menggunakan crawler untuk mengindeks konten halaman web dan membuatnya tersedia dalam hasil pencarian.
- Web Scraping: Crawling digunakan untuk mengekstrak data tertentu dari halaman web, misalnya untuk riset pasar atau pengumpulan data produk di situs e-commerce.
- Monitoring Situs Web: Crawlers bisa digunakan untuk memantau perubahan konten pada situs web, misalnya untuk melihat perubahan harga produk atau ketersediaan informasi baru.
  
 Contoh Crawler:
- Googlebot: Crawler yang digunakan oleh Google untuk mengindeks halaman-halaman web dan meng-update database mesin pencari mereka.
- Scrapy: Sebuah framework open-source yang sering digunakan untuk melakukan crawling dan scraping data dari situs web.

Tantangan dalam Crawling:
- Robots.txt: Banyak situs web yang menggunakan file `robots.txt` untuk memberikan instruksi kepada crawler tentang halaman mana yang boleh atau tidak boleh diakses.
- Kinerja & Bandwidth: Proses crawling dapat membebani server web jika dilakukan secara berlebihan atau tidak terkendali.
- Situs Dinamis: Banyak situs modern menggunakan JavaScript untuk memuat konten secara dinamis, yang dapat mempersulit crawling tradisional yang hanya mengunduh HTML statis.

Crawling adalah bagian fundamental dari cara internet bekerja, terutama dalam hal pencarian informasi secara otomatis dan sistematis.

In [1]:
!pip install requests beautifulsoup4 lxml pandas


1. requests

Pustaka requests digunakan untuk mengirimkan permintaan HTTP ke server dan menerima respons berupa halaman web (biasanya dalam format HTML). Ini adalah langkah pertama dalam web scraping untuk mengakses konten dari situs web.

Kegunaan:

Mengirimkan permintaan GET atau POST ke situs web.
Mendapatkan konten halaman web dalam format HTML atau JSON.

2. BeautifulSoup
Pustaka BeautifulSoup dari library bs4 digunakan untuk mem-parsing (mengurai) dan mengekstrak informasi dari HTML atau XML. Ini memudahkan penanganan elemen-elemen web seperti tag <div>, <a>, <table>, dll.

Kegunaan:

Memahami dan menavigasi struktur dokumen HTML.
Mengambil informasi tertentu (seperti teks, tabel, gambar, link, dll.) berdasarkan elemen HTML dan atributnya.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:


# URL halaman utama dari situs berita (misal: halaman kategori atau halaman utama berita)
url = 'https://sport.detik.com/sepakbola'

# Header untuk menghindari pemblokiran bot
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Fungsi untuk melakukan crawling halaman berita
def crawl_news(url):
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Gagal mengakses halaman:", url)
        return None

    soup = BeautifulSoup(response.text, 'lxml')

    # Struktur halaman situs berita dapat bervariasi, pastikan sesuai dengan situs target
    articles = soup.find_all('article')  # Atau elemen lain yang memuat berita
    news_data = []

    # Mengumpulkan data dari setiap artikel
    for article in articles:
        title_tag = article.find('h2', class_='title')  # Tag dan class sesuai situs yang di-crawl
        if title_tag:
            title = title_tag.text.strip()
            link = article.find('a')['href']
            date = article.find('time').text.strip()  # Sesuaikan jika berbeda

            # Menambahkan ke list
            news_data.append({
                'title': title,
                'url': link,
                'date': date
            })

    return news_data

# Fungsi untuk mengambil isi dari halaman detail berita
def get_news_content(news_url):
    response = requests.get(news_url, headers=headers)
    if response.status_code != 200:
        print(f"Gagal mengakses halaman berita: {news_url}")
        return ""

    soup = BeautifulSoup(response.text, 'lxml')

    # Temukan elemen yang berisi isi berita
    content = soup.find('div', class_='article-body')  # Sesuaikan dengan struktur HTML situs
    if content:
        paragraphs = content.find_all('p')
        news_content = "\n".join([p.text.strip() for p in paragraphs])
        return news_content
    return "Tidak ada konten yang ditemukan."

# Inisialisasi list untuk menyimpan semua berita
all_news = []

# Loop untuk mengambil beberapa halaman (misal, 5 halaman, asumsikan tiap halaman memuat 10 berita)
for i in range(1, 6):  # Loop untuk halaman 1 hingga 5
    page_url = f'{url}?page={i}'  # Sesuaikan parameter pagination situs target
    news_on_page = crawl_news(page_url)
    if news_on_page:
        for news in news_on_page:
            news_url = news['url']
            content = get_news_content(news_url)
            news['content'] = content
            all_news.append(news)

# Membatasi hasil ke 50 berita
all_news = all_news[:50]

# Menyimpan hasil crawling ke dalam DataFrame
df = pd.DataFrame(all_news)

# Menampilkan data
print(df.head())

# Menyimpan data ke CSV
df.to_csv('crawled_news_with_content.csv', index=False)
print("Data disimpan ke file crawled_news_with_content.csv")


Empty DataFrame
Columns: []
Index: []
Data disimpan ke file crawled_news_with_content.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')